This function computes the outermost vortex boundary $ \mathcal{B}(t_0) $ encircling vortex center $ \mathcal{C}({t_0}) $ derived from the LAVD-field.

The algorithm proceeds as follows:

* Detect initial positions $ \mathcal{C}(t_0) $ of vortex centres as local maxima of $ \mathrm{LAVD}_{t_0}^{t_N} $.
* Seek initial vortex boundaries $ \mathcal{B}(t_0) $ as outermost closed contours of $ \mathrm{LAVD}_{t_0}^{t_N}(\mathbf{x}_0) $ satisfying all of the following condtions:
    
    1. $ \mathcal{B}(t_0) $ encircles a vortex center $ \mathcal{C}(t_0) $
    2. $ \mathcal{B}(t_0) $ has arclength exceeding a threshold $ l_{min} $
    3. $ \mathcal{B}(t_0) $ has convexity deficiency below $ c_{d} $

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | $ X $ | 
| Y | array (Ny, Nx) | $ Y $|
| LAVD | array (Ny, Nx) | $ \mathrm{LAVD}_{t_0}^{t_N} $|
| distance | float | minimum distance between local maxima |
| n | float | resolution of contours |
| $ c_d $ | float | convexity deficiency |
| l_min | float | minimum eddy perimeter |
| loc_threshold | float | threshold on local maxima |
| Ncores | float | Number of cores used for parallel computing |
| vortex | list | list containing vortex boundaries |

In [1]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path in order to access the functions
sys.path.append(parent_directory+"/subfunctions/utils")

In [2]:
# import numpy
import numpy as np

# Import package to compute level set
from skimage import measure

# function which computes local maxima
from ipynb.fs.defs.loc_max import _loc_max

# import function to calculate area of closed curve
from shapely.geometry import Polygon, Point

# import function to compute convex hull of polygon
from scipy.spatial import ConvexHull

# Import package for progress bar
from tqdm.notebook import tqdm

# import library for parallel computing
from joblib import Parallel, delayed

In [3]:
def find_outermost_contour(X, Y, LAVD, distance, n, c_d, l_min, loc_threshold, Ncores):
    '''
    The outermost nearly convex contours from the Lagrangian averaged Vorticity Deviation (LAVD) 
    are extracted by specifying the maximum allowed convexity deficiendy "c_d" and minimum length "l_min"
    
    Parameters:
        X:              array (NY, NX)  X-meshgrid
        Y:              array (NY, NX)  Y-meshgrid 
        LAVD:           array(Ny, Nx), LAVD-field
        n:              float, resolution of contour increments
        c_d:            float, convexity deficiency: generally in the interval [10^(-6), 10^{-3}]
        l_min:          float, minimum length of vortex boundary
        loc_threshold:  float, local threshold on LAVD to find local maxima
        Ncores:         int, number of cores to be used for parallel computing
        
    Returns:
        LAVD: list(N,), list of vortices
    '''
    
    # compute local minimum of LAVD
    min_LAVD = np.nanmin(LAVD) # float
    
    # find local maxima in LAVD
    idx_x, idx_y, loc_max_x, loc_max_y, loc_max_field = _loc_max(distance, X, Y, LAVD, loc_threshold)
    
    # define grid spacing
    dx = X[0,1]-X[0,0] # float
    dy = Y[1,0]-Y[0,0] # float
    
    # iterate over all local maxima and find outermost level set 
    # of LAVD satisfying certain conditions listed above   
    def parallel_iteration(i):
        
        # initialize vortex to np.nan
        B = [np.nan, np.nan] # list (2, )
        
        # Break statement for loops
        BREAK = False
        
        # Point object: local maximum
        C = Point(X[idx_y[i], idx_x[i]], Y[idx_y[i], idx_x[i]])
        
        # iterate over level sets
        for j in np.linspace(min_LAVD, loc_max_field[i], n):
            
            # extract the x_0(\lambda,\phi_0)
            contour = measure.find_contours(LAVD, j) # list
            
            # iterate over contours associated to level set j
            for c in contour:
                
                if c.shape[0] <= 4:
                    break
                
                # coordinates of contour
                x_polygon = np.min(X) + c[:, 1]*dx # array
                y_polygon = np.min(Y) + c[:, 0]*dy # array
                
                # create polygon object
                polygon = Polygon(np.array([x_polygon,y_polygon]).T) # Polygon object
            
                # check if local maximum is inside contour and if polygon is closed
                if polygon.contains(C) and c[0,1] == c[-1,1] and c[0,0]==c[-1,0]:
                
                    # create convex hull
                    convex = ConvexHull(np.array([x_polygon,y_polygon]).T)

                    # Area of convex polygon 
                    # (convex.volume returns the area, whereas convex.area returns the length of the perimeter in the two dimensional case)
                    A_convex = convex.volume # float
            
                    # Area of polygon
                    A = polygon.area # float
            
                    # Length of polygon
                    L = polygon.length # float
            
                    # calculate convexity deficiency:
                    convexity_deficiency = abs((A_convex-A)/A_convex) # float
            
                    # if condition is satisfied --> break inner for loop as the vortex boundary associated to the local maximum has been found.
                    if L > l_min and convexity_deficiency < c_d:
                        B = [x_polygon, y_polygon]
                        BREAK = True
                        break
                        
            # Break out of second inner loop
            if BREAK:
                break
        
        return B
    
    # Compute vortex from LAVD with parallel computing
    vortex = Parallel(n_jobs=Ncores, verbose = 0)(delayed(parallel_iteration)(i) for i in tqdm(range(len(loc_max_field))))
    
    return vortex